### Install depends:

In [1]:
# Install TensorFlow (Kaggle default images usually have it pre-installed, but just in case)
!pip install -q tensorflow tqdm opencv-python-headless matplotlib tensorflow-addons


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 12.2 MB/s eta 0:00:00a 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydata-profiling 4.16.1 requires typeguard<5,>=3, but you have typeguard 2.13.3 which is incompatible.
inflect 7.5.0 requires typeguard>=4.0.1, but you have typeguard 2.13.3 which is incompatible.


### Train the TF model to predict mtg corner locations using the synthetic dataset:

In [2]:
import os
import json
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks
from tensorflow.keras.applications import MobileNetV2

# === Config ===
INPUT_DIR        = "/kaggle/input/mtg-keypoint-regression-dataset-5k"
ANNOTATION_FILE  = os.path.join(INPUT_DIR, "dataset", "dataset", "annotations.json")
IMAGE_DIR        = os.path.join(INPUT_DIR, "dataset", "dataset")
OUTPUT_DIR       = "/kaggle/working"
VISUALIZE_DIR    = os.path.join(OUTPUT_DIR, "training_visualizations")
MODEL_PATH       = os.path.join(OUTPUT_DIR, "mtg_keypoint_tfjs_safe_model")

IMAGE_SIZE       = 224
HEATMAP_SIZE     = 56
SIGMA_PIXELS     = 1.5
BATCH_SIZE       = 16
EPOCHS           = 35
ORIGINAL_DIM     = 1024

os.makedirs(VISUALIZE_DIR, exist_ok=True)

# === Load and validate annotations ===
with open(ANNOTATION_FILE, "r") as f:
    raw = json.load(f)

def valid(a):
    pts = a.get("corners", [])
    return (isinstance(pts, list) and len(pts) == 4 and
            all(isinstance(x, (int,float)) and isinstance(y, (int,float)) and
                0 <= x <= ORIGINAL_DIM and 0 <= y <= ORIGINAL_DIM
                for x,y in pts))

annotations = [a for a in raw if valid(a)]
split = int(0.9 * len(annotations))
train_anns, val_anns = annotations[:split], annotations[split:]

# === Precompute grid for heatmaps ===
xs = np.arange(HEATMAP_SIZE)
ys = np.arange(HEATMAP_SIZE)
grid_x, grid_y = np.meshgrid(xs, ys)

def make_heatmaps(corners):
    hm = np.zeros((HEATMAP_SIZE, HEATMAP_SIZE, 4), dtype=np.float32)
    for i, (x_o, y_o) in enumerate(corners):
        x = (x_o / ORIGINAL_DIM) * (HEATMAP_SIZE - 1)
        y = (y_o / ORIGINAL_DIM) * (HEATMAP_SIZE - 1)
        d2 = (grid_x - x)**2 + (grid_y - y)**2
        hm[..., i] = np.exp(-d2 / (2 * SIGMA_PIXELS**2))
    return hm

def data_gen(anns):
    while True:
        imgs = np.zeros((BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, 3), dtype=np.float32)
        hmts = np.zeros((BATCH_SIZE, HEATMAP_SIZE, HEATMAP_SIZE, 4), dtype=np.float32)
        coords = np.zeros((BATCH_SIZE, 4, 2), dtype=np.float32)
        for i in range(BATCH_SIZE):
            a = np.random.choice(anns)
            img = cv2.imread(os.path.join(IMAGE_DIR, a["filename"]))
            img = cv2.resize(img, (IMAGE_SIZE, IMAGE_SIZE)).astype(np.float32) / 255.0
            hm = make_heatmaps(a["corners"])
            imgs[i]   = img
            hmts[i]   = hm
            coords[i] = np.array(a["corners"], dtype=np.float32) / ORIGINAL_DIM
        yield imgs, (hmts, coords)

train_ds = tf.data.Dataset.from_generator(
    lambda: data_gen(train_anns),
    output_signature=(
        tf.TensorSpec((BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, 3), tf.float32),
        (tf.TensorSpec((BATCH_SIZE, HEATMAP_SIZE, HEATMAP_SIZE, 4), tf.float32),
         tf.TensorSpec((BATCH_SIZE, 4, 2), tf.float32))
    )
).prefetch(tf.data.AUTOTUNE)

val_ds = tf.data.Dataset.from_generator(
    lambda: data_gen(val_anns),
    output_signature=(
        tf.TensorSpec((BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, 3), tf.float32),
        (tf.TensorSpec((BATCH_SIZE, HEATMAP_SIZE, HEATMAP_SIZE, 4), tf.float32),
         tf.TensorSpec((BATCH_SIZE, 4, 2), tf.float32))
    )
).prefetch(tf.data.AUTOTUNE)

@tf.keras.utils.register_keras_serializable()
class SoftArgmax(layers.Layer):
    def __init__(self, H, W, **kw):
        super().__init__(**kw)
        xs = tf.linspace(0.0, 1.0, W)
        ys = tf.linspace(0.0, 1.0, H)
        gx, gy = tf.meshgrid(xs, ys)
        self.grid_x = tf.reshape(gx, [-1])
        self.grid_y = tf.reshape(gy, [-1])
        self.H, self.W = H, W

    def call(self, heatmaps):
        B = tf.shape(heatmaps)[0]
        flat = tf.reshape(heatmaps, [B, self.H*self.W, 4])
        probs = tf.nn.softmax(flat, axis=1)
        exp_x = tf.reduce_sum(probs * self.grid_x[None,:,None], axis=1)
        exp_y = tf.reduce_sum(probs * self.grid_y[None,:,None], axis=1)
        return tf.stack([exp_x, exp_y], axis=-1)

def build_model():
    inp = layers.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3), name="input_image")
    base = MobileNetV2(include_top=False, input_tensor=inp, weights='imagenet')
    x = base.output
    x = layers.Conv2DTranspose(256, 3, strides=2, padding='same', activation='relu', use_bias=False)(x)
    x = layers.Conv2D(128, 3, padding='same', activation='relu', use_bias=False)(x)
    x = layers.UpSampling2D(2)(x)
    x = layers.Conv2D(64, 3, padding='same', activation='relu', use_bias=False)(x)
    x = layers.UpSampling2D(2)(x)
    feat = layers.Conv2D(64, 3, padding='same', activation='relu', use_bias=False)(x)
    heatmaps = layers.Conv2D(4, 1, padding='same', activation=None, use_bias=False, name='heatmaps')(feat)
    coords = SoftArgmax(HEATMAP_SIZE, HEATMAP_SIZE, name='coords')(heatmaps)
    return models.Model(inputs=inp, outputs=[heatmaps, coords])

model = build_model()

def corner_error(y_true, y_pred):
    err = tf.norm((y_true - y_pred) * ORIGINAL_DIM, axis=-1)
    return tf.reduce_mean(err)

model.compile(
    optimizer=optimizers.Adam(1e-4),
    loss={'heatmaps': 'mse', 'coords': 'mse'},
    loss_weights={'heatmaps': 1.0, 'coords': 0.0},
    metrics={'coords': corner_error}
)

model.fit(
    train_ds,
    validation_data=val_ds,
    steps_per_epoch=len(train_anns) // BATCH_SIZE,
    validation_steps=len(val_anns) // BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=[
        callbacks.EarlyStopping(
            monitor='val_coords_corner_error',
            mode='min',
            patience=3,
            restore_best_weights=True
        )
    ]
)

model.export(MODEL_PATH)
print("✅ WASM-compatible SavedModel exported to:", MODEL_PATH)


2025-05-30 09:36:11.771428: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748597771.976407      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748597772.041382      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
I0000 00:00:1748597784.216745      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0
/tmp/ipykernel_31/113039195.py:109: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
 

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/35


I0000 00:00:1748597822.277904     100 service.cc:148] XLA service 0x7d50d8005ca0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1748597822.278792     100 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1748597825.363894     100 cuda_dnn.cc:529] Loaded cuDNN version 90300
E0000 00:00:1748597830.495977     100 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1748597830.692051     100 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


  2/295 ━━━━━━━━━━━━━━━━━━━━ 17s 58ms/step - coords_corner_error: 413.6206 - coords_loss: 0.0818 - heatmaps_loss: 0.0000e+00 - loss: 0.0818   

I0000 00:00:1748597841.621407     100 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


295/295 ━━━━━━━━━━━━━━━━━━━━ 146s 312ms/step - coords_corner_error: 190.0127 - coords_loss: 0.0263 - heatmaps_loss: 0.0000e+00 - loss: 0.0263 - val_coords_corner_error: 89.3670 - val_coords_loss: 0.0077 - val_heatmaps_loss: 0.0000e+00 - val_loss: 0.0077
Epoch 2/35
295/295 ━━━━━━━━━━━━━━━━━━━━ 68s 230ms/step - coords_corner_error: 42.5504 - coords_loss: 0.0018 - heatmaps_loss: 0.0000e+00 - loss: 0.0018 - val_coords_corner_error: 54.1543 - val_coords_loss: 0.0036 - val_heatmaps_loss: 0.0000e+00 - val_loss: 0.0036
Epoch 3/35
295/295 ━━━━━━━━━━━━━━━━━━━━ 60s 202ms/step - coords_corner_error: 30.2585 - coords_loss: 8.5430e-04 - heatmaps_loss: 0.0000e+00 - loss: 8.5430e-04 - val_coords_corner_error: 52.1150 - val_coords_loss: 0.0047 - val_heatmaps_loss: 0.0000e+00 - val_loss: 0.0047
Epoch 4/35
295/295 ━━━━━━━━━━━━━━━━━━━━ 56s 190ms/step - coords_corner_error: 23.4626 - coords_loss: 5.1843e-04 - heatmaps_loss: 0.0000e+00 - loss: 5.1843e-04 - val_coords_corner_error: 36.1154 - val_coords_loss:

## Convert to tensorflowjs for use on the web:

In [3]:
!pip install tensorflowjs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.1/89.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.2
    Uninstalling packaging-24.2:
      Successfully uninstalled packaging-24.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.5.0 requires fsspec[http]<=2024.12.0,>=2023.1.0, but you have fsspec 2025.3.2 which is incompatible.
ydata-profiling 4.16.1 requires typeguard<5,>=3, but you have typeguard 2.13.3 which is incompatible.
nilearn 0.11.1 requires scikit-learn>=1.4.0, but you have scikit-learn 1.2.2 which is incompatible.
pandas-gbq 0.26.1 requires google-api-core<3.0.0dev,>=2.10.2, but you have google-api-core 1.34.1 which is incompatible.
bigframes 1.36.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 whic

### Convert the Keras model to tfjs graph model for use on the web:

In [4]:
#!/usr/bin/env python3

import os
import shutil

# ─── CONFIG ──────────────────────────────────────────────────────────────────────
SAVEDMODEL_DIR = "/kaggle/working/mtg_keypoint_tfjs_safe_model"  # No .h5 here!
TFJS_OUT_DIR   = "/kaggle/working/web_model"

# ─── CLEAN ───────────────────────────────────────────────────────────────────────
if not os.path.isdir(SAVEDMODEL_DIR):
    raise FileNotFoundError(f"❌ SavedModel not found at: {SAVEDMODEL_DIR}")

shutil.rmtree(TFJS_OUT_DIR, ignore_errors=True)
os.makedirs(TFJS_OUT_DIR, exist_ok=True)

# ─── CONVERT TO TFJS ─────────────────────────────────────────────────────────────
print("🚀 Converting SavedModel to TensorFlow.js format...")
exit_code = os.system(
    f"tensorflowjs_converter "
    f"--input_format=tf_saved_model "
    f"--output_format=tfjs_graph_model "
    f"--strip_debug_ops=true "
    f"{SAVEDMODEL_DIR} "
    f"{TFJS_OUT_DIR}"
)

if exit_code != 0:
    raise RuntimeError(f"❌ TFJS conversion failed with exit code {exit_code}")

print(f"✅ TFJS GraphModel exported to: {TFJS_OUT_DIR}")

🚀 Converting SavedModel to TensorFlow.js format...


2025-05-30 09:52:35.743873: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748598755.764548     193 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748598755.770743     193 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
🌲 Try https://ydf.readthedocs.io, the successor of TensorFlow Decision Forests with more features and faster training!
I0000 00:00:1748598764.150368     193 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 12677 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0
I0000 00:00:1748598766.868865     193 devices.cc:67] Number of eligible GPUs (core co

✅ TFJS GraphModel exported to: /kaggle/working/web_model


In [5]:
import shutil
import os

# Paths
web_model_dir = os.path.join('/kaggle/working', 'web_model')
zip_path = os.path.join('/kaggle/working', 'web_model')

# Remove existing archive
if os.path.exists(f"{zip_path}.zip"):
    os.remove(f"{zip_path}.zip")

# Create ZIP archive
shutil.make_archive(zip_path, 'zip', web_model_dir)

print(f"✅ Created archive: {zip_path}.zip")


✅ Created archive: /kaggle/working/web_model.zip
